In [1]:
import os
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import torch
import torch.nn as nn
import torch.nn.functional as F

import torch_geometric
from torch_geometric.datasets import CoraFull, Planetoid, CitationFull
from torch_geometric.transforms import NormalizeFeatures
import torch_geometric.nn as gnn 

from models import GAT
from utils import train_model, test_model

torch.manual_seed(42)
torch.cuda.manual_seed(42)
np.random.seed(42)
random.seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/home/jaewoo/anaconda3/envs/graphormer/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = CitationFull(root='dataset/Cora', name='Cora', transform=NormalizeFeatures())

In [2]:
dataset = Planetoid(root='dataset/Planetoid', name='Cora', transform=NormalizeFeatures())

In [3]:
data = dataset[0]
df = pd.DataFrame(data.x)
df['y'] = data.y
train, valid = train_test_split(df, stratify=df.y, test_size=0.33)
data.train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
data.train_mask[train.index]=True

In [4]:
model=GAT(in_channels=dataset.num_features, hidden_channels=128, number_of_classes=dataset.num_classes, num_of_hidden_layers=2, device=device)
print(model.get_n_params())
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)

201479


In [5]:
for epoch in range(500):
    loss = train_model(model, data, optimizer, criterion)
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

Epoch: 000, Loss: 1.9577
Epoch: 001, Loss: 1.9536
Epoch: 002, Loss: 1.9495
Epoch: 003, Loss: 1.9453
Epoch: 004, Loss: 1.9407
Epoch: 005, Loss: 1.9358
Epoch: 006, Loss: 1.9304
Epoch: 007, Loss: 1.9245
Epoch: 008, Loss: 1.9181
Epoch: 009, Loss: 1.9111
Epoch: 010, Loss: 1.9035
Epoch: 011, Loss: 1.8953
Epoch: 012, Loss: 1.8866
Epoch: 013, Loss: 1.8773
Epoch: 014, Loss: 1.8676
Epoch: 015, Loss: 1.8575
Epoch: 016, Loss: 1.8472
Epoch: 017, Loss: 1.8370
Epoch: 018, Loss: 1.8270
Epoch: 019, Loss: 1.8174
Epoch: 020, Loss: 1.8086
Epoch: 021, Loss: 1.8009
Epoch: 022, Loss: 1.7942
Epoch: 023, Loss: 1.7888
Epoch: 024, Loss: 1.7844
Epoch: 025, Loss: 1.7805
Epoch: 026, Loss: 1.7768
Epoch: 027, Loss: 1.7726
Epoch: 028, Loss: 1.7677
Epoch: 029, Loss: 1.7619
Epoch: 030, Loss: 1.7553
Epoch: 031, Loss: 1.7482
Epoch: 032, Loss: 1.7409
Epoch: 033, Loss: 1.7335
Epoch: 034, Loss: 1.7261
Epoch: 035, Loss: 1.7187
Epoch: 036, Loss: 1.7112
Epoch: 037, Loss: 1.7034
Epoch: 038, Loss: 1.6952
Epoch: 039, Loss: 1.6864


In [6]:
test_acc, f1 = test_model(model, data)
print(f'Test Accuracy: {test_acc:.4f}')

              precision    recall  f1-score   support

           0       0.78      0.86      0.82       116
           1       0.91      0.69      0.79        72
           2       0.91      0.93      0.92       138
           3       0.85      0.87      0.86       270
           4       0.92      0.87      0.89       141
           5       0.83      0.82      0.82        98
           6       0.79      0.85      0.82        59

    accuracy                           0.86       894
   macro avg       0.86      0.84      0.85       894
weighted avg       0.86      0.86      0.86       894

Test Accuracy: 0.8568
